# Integration (`scipy.integrate`)

In [1]:
import scipy.integrate as integrate
import scipy.special as special
result = integrate.quad(lambda x: special.jv(2.5,x), 0, 4.5)
result

(1.1178179380783253, 7.866317216380692e-09)

In [2]:
from numpy import sqrt, sin, cos, pi
I = sqrt(2/pi)*(18.0/27*sqrt(2)*cos(4.5) - 4.0/27*sqrt(2)*sin(4.5) +
                sqrt(2*pi) * special.fresnel(3/sqrt(pi))[0])
I

1.117817938088701

In [3]:
print(abs(result[0]-I))

1.0375700298936863e-11


In [4]:
from scipy.integrate import quad
def integrand(x, a, b):
    return a*x**2 + b

a = 2
b = 1
I = quad(integrand, 0, 1, args=(a,b))
I

(1.6666666666666667, 1.8503717077085944e-14)

In [5]:
from scipy.integrate import quad
def integrand(t, n, x):
    return np.exp(-x*t) / t**n

In [6]:
def expint(n, x):
    return quad(integrand, 1, np.inf, args=(n, x))[0]

In [7]:
import numpy as np
vec_expint = np.vectorize(expint)

In [8]:
vec_expint(3, np.arange(1.0, 4.0, 0.5))

import scipy.special as special
special.expn(3, np.arange(1.0,4.0,0.5))

array([0.10969197, 0.05673949, 0.03013338, 0.01629537, 0.00893065,
       0.00494538])

In [9]:
result = quad(lambda x: expint(3, x), 0, np.inf)
print(result)

(0.33333333325010883, 2.8604069921197956e-09)


In [10]:
I3 = 1.0/3.0
print(I3)

0.3333333333333333


In [11]:
print(I3 - result[0])

8.322448286079975e-11


In [12]:
from scipy.integrate import quad, dblquad
def I(n):
    return dblquad(lambda t, x: np.exp(-x*t)/t**n, 0, np.inf, lambda x: 1, lambda x: np.inf)

print(I(4))

print(I(3))

print(I(2))

(0.2500000000043577, 1.2983033469368098e-08)
(0.33333333325010883, 1.3888461883425516e-08)
(0.4999999999985751, 1.3894083651858995e-08)


In [13]:
from scipy.integrate import dblquad
area = dblquad(lambda x, y: x*y, 0, 0.5, lambda x: 0, lambda x: 1-2*x)
area

(0.010416666666666668, 4.101620128472366e-16)

In [14]:
from scipy import integrate
N = 5
def f(t, x):
    return np.exp(-x*t) / t**N

integrate.nquad(f, [[1, np.inf],[0, np.inf]])

(0.2000000000189363, 1.3682975855986131e-08)

In [15]:
def f(x, y):
    return x*y

def bounds_y():
    return [0, 0.5]

def bounds_x(y):
    return [0, 1-2*y]

integrate.nquad(f, [bounds_x, bounds_y])

(0.010416666666666668, 4.101620128472366e-16)

In [16]:
def f1(x):
    return x**2

def f2(x):
    return x**3

x = np.array([1,3,4])
y1 = f1(x)
from scipy.integrate import simps
I1 = simps(y1, x)
print(I1)

21.0


In [17]:
y2 = f2(x)
I2 = integrate.simps(y2, x)
print(I2)

61.5


In [18]:
from scipy.integrate import solve_ivp
from scipy.special import gamma, airy
y1_0 = +1 / 3**(2/3) / gamma(2/3)
y0_0 = -1 / 3**(1/3) / gamma(1/3)
y0 = [y0_0, y1_0]
def func(t, y):
    return [t*y[1],y[0]]

t_span = [0, 4]
sol1 = solve_ivp(func, t_span, y0)
print("sol1.t: {}".format(sol1.t))
print("sol1.y[1]: {}".format(sol1.y[1]))
print("airy(sol.t)[0]:  {}".format(airy(sol1.t)[0]))

sol1.t: [0.         0.10097672 1.04643602 1.91060117 2.49872472 3.08684827
 3.62692846 4.        ]
sol1.y[1]: [0.35502805 0.328952   0.12801343 0.04008508 0.01601291 0.00623879
 0.00356316 0.00405982]
airy(sol.t)[0]:  [0.35502805 0.328952   0.12804768 0.03995804 0.01575943 0.00562799
 0.00201689 0.00095156]


In [19]:
rtol, atol = (1e-8, 1e-8)
sol2 = solve_ivp(func, t_span, y0, rtol=rtol, atol=atol)
print("sol2.y[1][::6]: {}".format(sol2.y[1][0::6]))
print("airy(sol2.t)[0][::6]: {}".format(airy(sol2.t)[0][::6]))

sol2.y[1][::6]: [0.35502805 0.19145234 0.06368989 0.0205917  0.00554734 0.00106409]
airy(sol2.t)[0][::6]: [0.35502805 0.19145234 0.06368989 0.0205917  0.00554733 0.00106406]


In [20]:
t = np.linspace(0, 4, 100)
sol3 = solve_ivp(func, t_span, y0, t_eval=t)

In [21]:
def gradient(t, y):
    return [[0,t], [1,0]]
sol4 = solve_ivp(func, t_span, y0, method='Radau', jac=gradient)

In [22]:
def G(u, v, f, k):
    return f * (1 - u) - u*v**2

def H(u, v, f, k):
    return -(f + k) * v + u*v**2

def grayscott1d(y, t, f, k, Du, Dv, dx):
    """
    Differential equations for the 1-D Gray-Scott equations.

    The ODEs are derived using the method of lines.
    """
    # The vectors u and v are interleaved in y.  We define
    # views of u and v by slicing y.
    u = y[::2]
    v = y[1::2]

    # dydt is the return value of this function.
    dydt = np.empty_like(y)

    # Just like u and v are views of the interleaved vectors
    # in y, dudt and dvdt are views of the interleaved output
    # vectors in dydt.
    dudt = dydt[::2]
    dvdt = dydt[1::2]

    # Compute du/dt and dv/dt.  The end points and the interior points
    # are handled separately.
    dudt[0]    = G(u[0],    v[0],    f, k) + Du * (-2.0*u[0] + 2.0*u[1]) / dx**2
    dudt[1:-1] = G(u[1:-1], v[1:-1], f, k) + Du * np.diff(u,2) / dx**2
    dudt[-1]   = G(u[-1],   v[-1],   f, k) + Du * (- 2.0*u[-1] + 2.0*u[-2]) / dx**2
    dvdt[0]    = H(u[0],    v[0],    f, k) + Dv * (-2.0*v[0] + 2.0*v[1]) / dx**2
    dvdt[1:-1] = H(u[1:-1], v[1:-1], f, k) + Dv * np.diff(v,2) / dx**2
    dvdt[-1]   = H(u[-1],   v[-1],   f, k) + Dv * (-2.0*v[-1] + 2.0*v[-2]) / dx**2

    return dydt

In [23]:
from scipy.integrate import odeint

y0 = np.random.randn(5000)
t = np.linspace(0, 50, 11)
f = 0.024
k = 0.055
Du = 0.01
Dv = 0.005
dx = 0.02

In [24]:
sola = odeint(grayscott1d, y0, t, args=(f, k, Du, Dv, dx))

In [25]:
solb = odeint(grayscott1d, y0, t, args=(f, k, Du, Dv, dx), ml=2, mu=2)

In [26]:
np.allclose(sola, solb)

True